# 면세유가 및 일반유가 전처리

### import

In [ ]:
import os
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings(action='ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

### 데이터셋 input

In [ ]:
일반유가 = pd.read_csv('../data/유가/원본/13_21_주유소_지역별_평균판매가격.csv', encoding = 'CP949')
일반유가

In [ ]:
일반유가.info()

In [ ]:
실제면세유가 = pd.read_csv('../data/유가/원본/15_21_주유소_지역별_면세유평균판매가격.csv', encoding = 'CP949')
실제면세유가

### 면세유가 및 일반유가 전처리

In [ ]:
## 널값 interpolate 선형으로 보간

실제면세유가 = 실제면세유가.interpolate(method='values')

In [ ]:
## 일반유가로 면세유 가격 산정방식

def helper(x):
    return x - (x/11 + 528.75) 

계산면세유가 = 일반유가[일반유가.columns[1:]].apply(lambda x : helper(x))
계산면세유가

In [ ]:
## 계산한 면세유가와 실제면세유가 간 오차계산

오차 = 실제면세유가.drop('구분', axis = 1) - 계산면세유가.iloc[1049:].reset_index(drop = True)
오차

In [ ]:
## 지역별 평균 오차(지역별 주요소 전체기간 평균 마진 금액) 산정

평균오차 = 오차.describe().loc['mean']
평균오차

In [ ]:
## 계산면세유가에 평균주요소 마진 합산하여 면세가격 채울 부분 산정

for idx in 계산면세유가.index :
    계산면세유가.iloc[idx] += 평균오차

In [ ]:
## 최종 게산면세유가

계산면세유가

In [ ]:
## 날짜 삽입

계산면세유가['구분'] = 일반유가['구분']
계산면세유가

In [ ]:
## 실제면세유 가격에 필요한 기간(2013~2015) 합산

최종면세유가 = pd.concat([계산면세유가[:1049],실제면세유가]).reset_index(drop = True)
최종면세유가

In [ ]:
## 컬럼 정리

최종면세유가 = 최종면세유가[['구분','서울', '부산', '대구', '인천', '광주', '대전', '울산', '경기', '강원', '충북', '충남', '전북','전남', '경북', '경남', '제주', '세종']]
최종면세유가

In [ ]:
## 날짜 형식 변경

최종면세유가['구분'] = pd.to_datetime(최종면세유가['구분'], format='%Y년%m월%d일')
최종면세유가

### 데이터셋 파일 저장

In [ ]:
최종면세유가.rename(columns = {'구분':'날짜'}, inplace = True)
최종면세유가.to_csv(path_or_buf='../data/유가/13_21_면세유가.csv', encoding= 'utf-8-sig')